In [1]:
!pip install transformers
!pip install nltk
!pip install sentencepiece
!pip install pytorch-crf

     |████████████████████████████████| 2.2MB 18.6MB/s 
     |████████████████████████████████| 3.3MB 53.8MB/s 
     |████████████████████████████████| 870kB 46.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=b7925441aeb99b3e7090fc7e5d6879e353ee857e441f89a93d19de9d95dafaca
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 20.3MB/s 


In [2]:
!pip install scispacy==0.3.0
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_core_sci_sm-0.3.0.tar.gz

     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 10.4MB 12.3MB/s 
     |████████████████████████████████| 13.5MB 151kB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 1.1MB 51.3MB/s 
     |████████████████████████████████| 194kB 58.1MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 33.1MB 1.2MB/s 
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.3.0-cp37-none-any.whl size=33119280 sha256=58bd1ba1f76aee515a542e8344d867ce21c66271468092ef41be445dfbabe8f0
  Stored in directory: /root/.cache/pip/wheels/92/d2/d2/5aef870612ebd315c4b7b191f7ab5ff544281ef019d7c10c5d
Successfully built en-core-sci-sm


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers
from transformers import *
import spacy
import en_core_sci_sm
from google.colab import drive
import os
import re
from torchcrf import CRF

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

In [6]:
tokenizer

PreTrainedTokenizer(name_or_path='allenai/scibert_scivocab_uncased', vocab_size=31090, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [7]:
nlp = en_core_sci_sm.load()

In [8]:
def sent_tok(text):
  doc=nlp(text)
  return [s.text for s in doc.sents]

In [9]:
drive.mount("/content/drive/")
train_files=os.listdir("/content/drive/MyDrive/MeasEval/final_data/train/text")
val_files=os.listdir("/content/drive/MyDrive/MeasEval/final_data/dev/text")
print(train_files)

Mounted at /content/drive/
['S016412121300188X-5066.txt', 'S0016236113008041-3269.txt', 'S0019103512002801-1927.txt', 'S0019103512004009-4492.txt', 'S0012821X13007309-1509.txt', 'S0012821X12004384-1599.txt', 'S2211124713006475-1195.txt', 'S0021979713004438-2148.txt', 'S0019103512003995-2760.txt', 'S0019103512003533-5598.txt', 'S0019103512003533-3348.txt', 'S0012821X12004384-1302.txt', 'S0019103512004009-5507.txt', 'S0016236113008041-3161.txt', 'S1750583613004192-714.txt', 'S0378383912000130-3662.txt', 'S0016236113008041-3153.txt', 'S0019103512003995-3548.txt', 'S0019103512002801-1342.txt', 'S0022399913003358-1044.txt', 'S0019103513005058-4349.txt', 'S0165587612003680-953.txt', 'S0032386113009889-2123.txt', 'S1389128612002496-5994.txt', 'S0019103512004009-3488.txt', 'S0012821X12004384-1415.txt', 'S0022000014000026-18167.txt', 'S016412121300188X-4436.txt', 'S0019103513005058-1737.txt', 'S0019103511004994-1565.txt', 'S0021979713004438-1415.txt', 'S0025322712001600-2230.txt', 'S00162361130

In [10]:
print(len(train_files))
print(len(val_files))

278
20


In [11]:
ans = []
for file in train_files:
  path_txt=os.path.join("/content/drive/MyDrive/MeasEval/final_data/train/text/",file)
  path_tsv=os.path.join("/content/drive/MyDrive/MeasEval/final_data/train/tsv/",file[:-3]+"tsv")
  if(not(os.path.exists(path_tsv))):
    continue
  file_tsv=pd.read_csv(path_tsv,sep = '\t')
  tsv_list = file_tsv.to_numpy()
  with open(path_txt, 'r') as f:
    text=f.read()
  sentence_split = sent_tok(text=text)
  len_p = 0
  for sen in sentence_split:
    val = []
    val.append(sen)
    for i in range(len(tsv_list)):
      if tsv_list[i][2] == "Quantity" and tsv_list[i][3]>=len_p and tsv_list[i][4]<=len_p+len(sen):
        val.append((tsv_list[i][3] - len_p, tsv_list[i][4] - len_p))
    ans.append(val)
    len_p += len(sen)+1

In [12]:
ans[5]

['Additional case studies show that coherent clusters can help reveal the structure of a program and identify structural defects.']

In [13]:
ans_dev = []
for file in val_files:
  path_txt=os.path.join("/content/drive/MyDrive/MeasEval/final_data/dev/text/",file)
  path_tsv=os.path.join("/content/drive/MyDrive/MeasEval/final_data/dev/tsv/",file[:-3]+"tsv")
  if(not(os.path.exists(path_tsv))):
    continue
  file_tsv=pd.read_csv(path_tsv,sep = '\t')
  tsv_list = file_tsv.to_numpy()
  with open(path_txt, 'r') as f:
    text=f.read()
  sentence_split = sent_tok(text=text)
  len_p = 0
  for sen in sentence_split:
    val = []
    val.append(sen)
    for i in range(len(tsv_list)):
      if tsv_list[i][2] == "Quantity" and tsv_list[i][3]>=len_p and tsv_list[i][4]<=len_p+len(sen):
        val.append((tsv_list[i][3] - len_p, tsv_list[i][4] - len_p))
    ans_dev.append(val)
    len_p += len(sen)+1

In [14]:
ans_dev[6]

['The instrument is mounted on top of a movable turntable which rotates about the −y-axis of the spacecraft within 86 s, nominally.',
 (113, 128)]

In [15]:
!pip install stanza

     |████████████████████████████████| 286kB 23.5MB/s 


In [16]:
import stanza
stanza.download('en')
nlp2=stanza.Pipeline(lang='en', processors='tokenize', tokenize_no_ssplit=True)

2021-04-12 13:36:08 INFO: Downloading default packages for language: en (English)...
2021-04-12 13:37:32 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-04-12 13:37:32 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2021-04-12 13:37:32 INFO: Use device: gpu
2021-04-12 13:37:32 INFO: Loading: tokenize
2021-04-12 13:37:43 INFO: Done loading processors!


In [17]:
def length(text):
  l = 0
  for i in range(len(text)):
    if text[i] != '#':
      l += 1
  return l

In [18]:
pretrain = []
i = 0
for val in ans:
  sen_tok = []
  sen = nlp2(val[0])
  label_en = []
  for w in sen.sentences[0].words[:]:
    rng = re.findall(r'\d+', w.misc) 
    bert_tok = tokenizer.tokenize(w.text)
    sen_tok.extend(bert_tok)
    w_len = int(rng[0])
    for t in bert_tok:
      c  = 0
      for i in range(1, len(val)):
        if val[i][0] <= w_len and val[i][1] >= w_len + length(t):
          c += 1
      if c != 0:
        label_en.append(1)
      else:
        label_en.append(0)
      w_len += length(t)
  pretrain.append([sen_tok, label_en])

In [19]:
pretrain[2]

[['this',
  'paper',
  'extends',
  'previous',
  'work',
  'in',
  'many',
  'ways',
  ',',
  'firstly',
  'by',
  'introducing',
  'an',
  'efficient',
  'hashing',
  'algorithm',
  'for',
  'slice',
  'approximation',
  '.'],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [20]:
validation = []
i = 0
for val in ans_dev:
  sen_tok = []
  sen = nlp2(val[0])
  label_en = []
  for w in sen.sentences[0].words[:]:
    rng = re.findall(r'\d+', w.misc) 
    bert_tok = tokenizer.tokenize(w.text)
    sen_tok.extend(bert_tok)
    w_len = int(rng[0]);
    for t in bert_tok:
      c  = 0
      for i in range(1, len(val)):
        if val[i][0] <= w_len and val[i][1] >= w_len + length(t):
          c += 1
      if c != 0:
        label_en.append(1)
      else:
        label_en.append(0)
      w_len += length(t)
  validation.append([sen_tok, label_en])

In [21]:
validation[2]

[['low',
  '-',
  'energy',
  'electrons',
  'and',
  'ions',
  'measured',
  'in',
  'le',
  'are',
  'separated',
  'by',
  'a',
  'strong',
  'internal',
  'magnet',
  '.'],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [22]:
train = []
for val in pretrain:
  lab = np.zeros(256)
  tok_arr = np.zeros(256)
  att_mask = np.zeros(256)
  sen_tok = []
  tok_arr[0] = 102
  att_mask[0] = 1
  for tok in val[0]:
    sen_tok.append(tokenizer.convert_tokens_to_ids(tok))
  for i in range(len(val[1])):
    lab[i+1] = val[1][i]
    tok_arr[i+1] = sen_tok[i]
    att_mask[i+1] = 1
  train.append([tok_arr,att_mask, lab])


In [23]:
train[2]

[array([  102.,   238.,  1203., 10615.,  1061.,   697.,   121.,  1164.,
         4568.,   422.,  9693.,   214.,  9344.,   130.,  3316., 29054.,
         1172.,   168., 11735.,  4528.,   205.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0., 

In [24]:
val_data = []
for val in validation:
  lab = np.zeros(256)
  tok_arr = np.zeros(256)
  att_mask = np.zeros(256)
  sen_tok = []
  tok_arr[0] = 102
  att_mask[0] = 1
  for tok in val[0]:
    sen_tok.append(tokenizer.convert_tokens_to_ids(tok))
  for i in range(len(val[1])):
    lab[i+1] = val[1][i]
    tok_arr[i+1] = sen_tok[i]
    att_mask[i+1] = 1
  val_data.append([tok_arr,att_mask, lab])

In [25]:
x_train_id = np.zeros((0,256))
x_train_mask = np.zeros((0,256))
y_train = np.zeros((0,256))
for val in train:
  x_train_id = np.vstack((x_train_id, val[0]))
  x_train_mask = np.vstack((x_train_mask, val[1]))
  y_train = np.vstack((y_train, val[2]))

In [26]:
x_val_id = np.zeros((0,256))
x_val_mask = np.zeros((0,256))
y_val = np.zeros((0,256))
for val in val_data:
  x_val_id = np.vstack((x_val_id, val[0]))
  x_val_mask = np.vstack((x_val_mask, val[1]))
  y_val = np.vstack((y_val, val[2]))

In [27]:
for i in range(len(y_train)):
  for j in range(1,len(y_train[0])):
    if y_train[i][j-1] == 0 and y_train[i][j] == 1:
      y_train[i][j] = 2
for i in range(len(y_val)):
  for j in range(1,len(y_val[0])):
    if y_val[i][j-1] == 0 and y_val[i][j] == 1:
      y_val[i][j] = 2

In [28]:
from torch.utils.data import TensorDataset,DataLoader

train_data = TensorDataset(torch.from_numpy(x_train_id), torch.from_numpy(x_train_mask), torch.from_numpy(y_train))
val_data = TensorDataset(torch.from_numpy(x_val_id), torch.from_numpy(x_val_mask), torch.from_numpy(y_val))

In [29]:
batch_size = 12
train_loader = DataLoader(train_data, shuffle=True, batch_size = batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size = batch_size)


In [30]:
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')

In [31]:

class BERT_Arch(nn.Module):

    def __init__(self, bert, embed_dim, hidden_dim, drop_prob, n_layers, out_dim):
      
      super(BERT_Arch, self).__init__()
      self.bert = bert 
      self.dropout = nn.Dropout(drop_prob)
      self.fc1 = nn.Linear(2*embed_dim,out_dim)
      self.w1 = nn.Linear(embed_dim, embed_dim)
      self.w2 = nn.Linear(embed_dim, embed_dim)
      #self.bilstm = nn.LSTM(embed_dim, hidden_dim,  bidirectional=True, batch_first=True)
      self.softmax = nn.LogSoftmax(dim = 2)
      self.crf = CRF(3, batch_first=True)  
      self.tanh = nn.Tanh()

    #define the forward pass
    def forward(self, sent_id, mask_val, labels=None):
      x = self.bert(sent_id, attention_mask=mask_val)
      x = x.last_hidden_state
      x = self.tanh(x)
      cls = x[:,0,:]
      cls = cls.unsqueeze(1).repeat(1, 256, 1)
      cls = self.w1(cls)
      x = self.w2(x)
      x = torch.cat([x,cls], dim = 2)
      #x,_ = self.bilstm(x)
      x = self.dropout(x)
      x = self.fc1(x)
      mask_val = mask_val.type(torch.uint8)
      logit = self.softmax(x)
      if labels is not None:
          loss = -self.crf(logit, labels, mask=mask_val, reduction='mean')
          return loss
      else:
          prediction = self.crf.decode(x, mask=mask_val)
          return prediction

In [32]:
bert_model = BERT_Arch(model, 768, 64, 0.1, 1,3)
bert_model = bert_model.to(device)

In [33]:
print(bert_model)

BERT_Arch(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

In [34]:
optimizer = torch.optim.SGD(bert_model.parameters(), lr=0.01,momentum=0.9)

In [35]:

def score(val_loader):
  p=0
  n=0
  pos=0
  neg=0
  prec_num = 0 
  prec_den = 0
  rec_num = 0
  rec_den = 0 
  p1 = 0
  n1=0
  for seq, mask, y in val_loader:
      bert_model.zero_grad()
      bert_model.eval()
      y_pred = bert_model(seq.long().to(device), mask.long().to(device))
      np_out = np.zeros((len(y_pred),256, 1))
      np_act = (y.cpu().data.numpy() >= 1).astype(int).reshape((len(y), len(y[0]),1))
      
      for i in range(np_out.shape[0]):
        prev_2 = 0
        for j in range(np_out.shape[1]):
          for k in range(np_out.shape[2]):
            if j >= len(y_pred[i]):
              np_out[i][j][k] = 0
            elif y_pred[i][j] == 2:
              np_out[i][j][k]=1
              prev_2 = 1
            elif y_pred[i][j] == 1 and prev_2 == 1:
              np_out[i][j][k]=1
            else:
              prev_2 = 0
              np_out[i][j][k]=0

      for i in range(np_out.shape[0]):
        for j in range(np_out.shape[1]):
          if np.max(np_out[i,j,:])==1 or np.max(np_act[i,j,:])==1:
            if np.max(np_out[i,j,:])==1 and np.max(np_act[i,j,:])==1:
              p1 = p1 + 1
            else:
              n1 = n1 + 1

      for i in range(np_out.shape[0]):
        for j in range(np_out.shape[1]):
          for k in range(np_out.shape[2]):
            if np_out[i,j,k] == np_act[i,j,k]:
              p = p +1
            else:
              n=n+1
            if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
              if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
                pos = pos + 1
              else:
                neg = neg + 1
            if np_out[i,j,k]==1:
              if np_out[i,j,k]==np_act[i,j,k]:
                prec_num = prec_num + 1
              else:
                prec_den = prec_den + 1
            if np_act[i,j,k]==1:
              if np_out[i,j,k]==np_act[i,j,k]:
                rec_num = rec_num + 1
              else:
                rec_den = rec_den + 1
  if prec_num+prec_den == 0:
    return 0
  precision = prec_num/(prec_num+prec_den)
  if rec_num+rec_den == 0:
    return 0
  recall = rec_num/(rec_num+rec_den)
  if precision == 0 and recall == 0:
    return 0
  F1 = 2*precision*recall/(precision+recall)
  return F1

In [36]:
epochs = 10
for e in range(epochs):
  
  bert_model.train()
  i=0
  train_loss=0
  for seq, mask, y in train_loader:
    bert_model.zero_grad()
    loss = bert_model(seq.long().to(device), mask.long().to(device), y.long().to(device))
    train_loss += loss.item()*batch_size
    loss.backward()
    torch.nn.utils.clip_grad_norm_(bert_model.parameters(), 1.0)
    optimizer.step()
    if(i%5==0):
      print("Epoch-{}/{} Iterations-{} loss-{}".format(e+1,epochs,i+1,loss.item()))
    i+=1
  
  bert_model.eval()
  f1_score = score(val_loader)
  val_loss=0
  for seq, mask, y in val_loader:
    bert_model.zero_grad()
    loss = bert_model(seq.long().to(device), mask.long().to(device), y.long().to(device))
    val_loss += loss.item()*batch_size
    i+=1
  
  print("Epoch-{}/{} train_loss-{} Val_loss-{} F1 Score-{}".format(e+1,epochs,train_loss/len(train_loader),val_loss/len(val_loader), f1_score))

Epoch-1/10 Iterations-1 loss-38.25678253173828
Epoch-1/10 Iterations-6 loss-20.052963256835938
Epoch-1/10 Iterations-11 loss-9.795960426330566
Epoch-1/10 Iterations-16 loss-16.86676597595215
Epoch-1/10 Iterations-21 loss-9.916542053222656
Epoch-1/10 Iterations-26 loss-8.461443901062012
Epoch-1/10 Iterations-31 loss-4.942470073699951
Epoch-1/10 Iterations-36 loss-4.820696830749512
Epoch-1/10 Iterations-41 loss-1.379876971244812
Epoch-1/10 Iterations-46 loss-2.8941097259521484
Epoch-1/10 Iterations-51 loss-4.564614772796631
Epoch-1/10 Iterations-56 loss-2.1300179958343506
Epoch-1/10 Iterations-61 loss-3.887444496154785
Epoch-1/10 Iterations-66 loss-1.0145280361175537
Epoch-1/10 Iterations-71 loss-5.293062210083008
Epoch-1/10 Iterations-76 loss-1.2408400774002075
Epoch-1/10 Iterations-81 loss-3.9129576683044434
Epoch-1/10 Iterations-86 loss-2.5822553634643555
Epoch-1/10 Iterations-91 loss-7.491784572601318
Epoch-1/10 Iterations-96 loss-0.9272007942199707
Epoch-1/10 Iterations-101 loss-2.7

In [37]:
save_path = '/content/drive/MyDrive/MeasEval/scibert_crf_quant_ent_2iter.pt'
torch.save(bert_model, save_path)

In [38]:
p=0
n=0
pos=0
neg=0
prec_num = 0 
prec_den = 0
rec_num = 0
rec_den = 0 
p1 = 0
n1=0
for seq, mask, y in val_loader:
    bert_model.zero_grad()
    bert_model.eval()
    y_pred = bert_model(seq.long().to(device), mask.long().to(device))
    np_out = np.zeros((len(y_pred),256, 1))
    np_act = (y.cpu().data.numpy() >= 1).astype(int).reshape((len(y), len(y[0]),1))
    
    for i in range(np_out.shape[0]):
      prev_2 = 0
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if j >= len(y_pred[i]):
            np_out[i][j][k] = 0
          elif y_pred[i][j] == 2:
            np_out[i][j][k]=1
            prev_2 = 1
          elif y_pred[i][j] == 1 and prev_2 == 1:
            np_out[i][j][k]=1
          else:
            prev_2 = 0
            np_out[i][j][k]=0

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        if np.max(np_out[i,j,:])==1 or np.max(np_act[i,j,:])==1:
          if np.max(np_out[i,j,:])==1 and np.max(np_act[i,j,:])==1:
            p1 = p1 + 1
          else:
            n1 = n1 + 1

    for i in range(np_out.shape[0]):
      for j in range(np_out.shape[1]):
        for k in range(np_out.shape[2]):
          if np_out[i,j,k] == np_act[i,j,k]:
            p = p +1
          else:
            n=n+1
          if (np_out[i,j,k])==1 or (np_act[i,j,k])==1:
            if (np_out[i,j,k])==1 and (np_act[i,j,k])==1:
              pos = pos + 1
            else:
              neg = neg + 1
          if np_out[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              prec_num = prec_num + 1
            else:
              prec_den = prec_den + 1
          if np_act[i,j,k]==1:
            if np_out[i,j,k]==np_act[i,j,k]:
              rec_num = rec_num + 1
            else:
              rec_den = rec_den + 1

precision = prec_num/(prec_num+prec_den)
recall = rec_num/(rec_num+rec_den)
F1 = 2*precision*recall/(precision+recall)

print("Entity recognition modified accuracy:-" + str(p1/(p1+n1)))
print("--------NER RESULTS--------")
print("Accuracy:-" + str(p/(n+p)))
print("Modified Accuracy:-" + str(pos/(pos+neg)))
print("Precision:-" + str(precision))
print("Recall:-" + str(recall))
print("F1 score:-"+str(F1))

Entity recognition modified accuracy:-0.9017199017199017
--------NER RESULTS--------
Accuracy:-0.99875
Modified Accuracy:-0.9017199017199017
Precision:-0.9362244897959183
Recall:-0.9607329842931938
F1 score:-0.9483204134366926
